# models behavior vs human behavior epileptic

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('svg')
matplotlib.rcParams['svg.fonttype'] = 'none'
import os
os.environ["OMP_NUM_THREADS"] = "1"
import seaborn as sns 
import ast
from scipy.stats import pearsonr
from statsmodels.stats.multitest import fdrcorrection
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import FixedLocator
from scipy.spatial.distance import euclidean
from scipy.cluster.hierarchy import linkage, leaves_list
from scipy.spatial.distance import squareform
import warnings
warnings.filterwarnings("ignore")
from scipy.spatial.distance import correlation
from scipy.stats import spearmanr
from scipy.stats import sem
from matplotlib.ticker import FixedLocator
from scipy.optimize import curve_fit
from scipy.optimize import OptimizeWarning
import warnings
from sklearn.metrics import mean_squared_error
import pathlib
import matplotlib.gridspec as gridspec

distributions_map = {"uniform": 0, "low": 1, "high": 2}
actions = {"arrowdown": 0, "arrowup": 1}

In [2]:
output_dir = r"29_RL_agent_TD_learn_models_behavior_across_models"
os.makedirs(output_dir, exist_ok=True)


folder_path_participants = 'data_risk_added_epileptic'

folder_path_greedy = '13_RL_agent_TDlearn_output/model_behavior'
folder_path_softmax = '13_RL_agent_TDlearn_output_softmax/model_behavior'
folder_path_dualQ = '13_RL_agent_TDlearn_output_risk_dualQ/model_behavior'
folder_path_rs = '13_RL_agent_TDlearn_output_risk_sensitive/model_behavior'
folder_path_wsls = '13_RL_agent_TDlearn_output_wsls/model_behavior'


df_participants = []
df_greedy = []
df_softmax = []
df_dualQ = []
df_rs = []
df_wsls = []



def find_matching_csv(folder_path, df_list):
            for csv_file in os.listdir(folder_path):
                if clean_name in csv_file and csv_file.endswith('.csv'):
                    csv_path = os.path.join(folder_path, csv_file)
                    df_csv = pd.read_csv(csv_path)
                    df_list.append(df_csv)





for file_name in os.listdir(folder_path_participants):
    if file_name.endswith('.csv'):
        file_path = os.path.join(folder_path_participants, file_name)
        df = pd.read_csv(file_path)
        df = df[df['outcome'].str.lower() != 'na'].reset_index(drop=True) 
        df_participants.append(df)

        clean_name = file_name.removeprefix("task_data_").removesuffix(".csv")


        find_matching_csv(folder_path_greedy, df_greedy)
        find_matching_csv(folder_path_softmax, df_softmax)
        find_matching_csv(folder_path_dualQ, df_dualQ)
        find_matching_csv(folder_path_rs, df_rs)
        find_matching_csv(folder_path_wsls, df_wsls)

In [3]:
num_participants = len(df_participants)
num_participants

9

In [4]:
for i in range(len(df_participants)):
    block = df_participants[i]['block']
    
    for df_list in [df_participants, df_wsls, df_greedy, df_softmax, df_dualQ, df_rs]:
        df_list[i]['block'] = block

In [5]:
for df in df_participants:
    df['block_type'] = None
    df.loc[df['block'] == 4, 'block_type'] = 'mix'              # block 4 is mix
    df.loc[df['block'].isin([1, 2, 3]), 'block_type'] = 'fix'   # else is fix

    # Convert totalReward column to float
    df['totalReward'] = df['totalReward'].astype(float)
    df['arrowRT'] = df['arrowRT'].astype(float)



    

for df in df_greedy:
    df['block_type'] = None
    df.loc[df['block'] == 4, 'block_type'] = 'mix'              # block 4 is mix
    df.loc[df['block'].isin([1, 2, 3]), 'block_type'] = 'fix'   # else is fix   

for df in df_softmax:
    df['block_type'] = None
    df.loc[df['block'] == 4, 'block_type'] = 'mix'              # block 4 is mix
    df.loc[df['block'].isin([1, 2, 3]), 'block_type'] = 'fix'   # else is fix


for df in df_dualQ:
    df['block_type'] = None
    df.loc[df['block'] == 4, 'block_type'] = 'mix'              # block 4 is mix
    df.loc[df['block'].isin([1, 2, 3]), 'block_type'] = 'fix'   # else is fix

for df in df_rs:
    df['block_type'] = None
    df.loc[df['block'] == 4, 'block_type'] = 'mix'              # block 4 is mix
    df.loc[df['block'].isin([1, 2, 3]), 'block_type'] = 'fix'   # else is fix

for df in df_wsls:
    df['block_type'] = None
    df.loc[df['block'] == 4, 'block_type'] = 'mix'              # block 4 is mix
    df.loc[df['block'].isin([1, 2, 3]), 'block_type'] = 'fix'   # else is fix




# adding myCard and youCard to the models dataset and also adding model_outcome

In [6]:
for i in range(len(df_participants)):
    myCard = df_participants[i]['myCard']
    yourCard = df_participants[i]['yourCard']
    distributions = df_participants[i]['distribution']
    block_type = df_participants[i]['block_type']
    
    for df_list in [df_participants, df_wsls, df_greedy, df_softmax, df_dualQ, df_rs]:
        df_list[i]['myCard'] = myCard
        df_list[i]['yourCard'] = yourCard
        df_list[i]['distribution'] = distributions
        df_list[i]['block_type'] = block_type

# adding model outcome

In [7]:
for df in df_wsls:
    outcomes = []
    for i in range(len(df)):
        my = df.loc[i, 'myCard']
        your = df.loc[i, 'yourCard']
        choice = df.loc[i, 'model_choices']
        
        if ((my > your and choice == 1) or (my < your and choice == 0)):
            outcomes.append('win')
        else:
            outcomes.append('lose')
    
    df['outcome'] = outcomes



for df in df_greedy:
    outcomes = []
    for i in range(len(df)):
        my = df.loc[i, 'myCard']
        your = df.loc[i, 'yourCard']
        choice = df.loc[i, 'model_choices']
        
        if ((my > your and choice == 1) or (my < your and choice == 0)):
            outcomes.append('win')
        else:
            outcomes.append('lose')
    
    df['outcome'] = outcomes
    
    


for df in df_softmax:
    outcomes = []
    for i in range(len(df)):
        my = df.loc[i, 'myCard']
        your = df.loc[i, 'yourCard']
        choice = df.loc[i, 'model_choices']
        
        if ((my > your and choice == 1) or (my < your and choice == 0)):
            outcomes.append('win')
        else:
            outcomes.append('lose')
    
    df['outcome'] = outcomes
    
    

for df in df_dualQ:
    outcomes = []
    for i in range(len(df)):
        my = df.loc[i, 'myCard']
        your = df.loc[i, 'yourCard']
        choice = df.loc[i, 'model_choices']
        
        if ((my > your and choice == 1) or (my < your and choice == 0)):
            outcomes.append('win')
        else:
            outcomes.append('lose')
    
    df['outcome'] = outcomes


for df in df_rs:
    outcomes = []
    for i in range(len(df)):
        my = df.loc[i, 'myCard']
        your = df.loc[i, 'yourCard']
        choice = df.loc[i, 'model_choices']
        
        if ((my > your and choice == 1) or (my < your and choice == 0)):
            outcomes.append('win')
        else:
            outcomes.append('lose')
    
    df['outcome'] = outcomes

In [8]:
df_participants[2]

,arrowRT,distribution,interTrialInterval,outcome,myCard,yourCard,spaceRT,totalReward,trialIndex,trialType,choice,block,timeoutRepeat,is_within_IQR,risk,block_type
0,2135.0,uniform,835,win,4,7,14079,10.5,0,response,arrowdown,1,0,0,0.375,fix
1,1203.0,uniform,926,win,9,4,1804,11.0,1,response,arrowup,1,0,1,0.000,fix
2,1035.0,uniform,934,win,1,9,766,11.5,2,response,arrowdown,1,0,1,0.000,fix
3,827.0,uniform,991,lose,2,1,1189,11.0,3,response,arrowdown,1,0,1,0.125,fix
4,1306.0,uniform,970,win,9,3,1887,11.5,4,response,arrowup,1,0,1,0.000,fix
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,939.0,uniform,773,win,4,6,802,87.0,24,response,arrowdown,4,0,1,0.375,mix
266,1210.0,high,811,win,6,8,1345,87.5,122,response,arrowdown,4,0,1,0.385,mix
267,275.0,high,838,win,3,5,254,88.0,123,response,arrowdown,4,0,1,0.071,mix
268,907.0,uniform,992,win,2,4,774,88.5,14,response,arrowdown,4,0,1,0.125,mix


In [9]:
df_greedy[2]

,model_choices,participant_choices,model_total_reward,participant_total_reward,q_val,block,block_type,myCard,yourCard,distribution,outcome
0,0,0,10.5,10.5,"[[[-0.010402418712398161, 0.013333949566327224...",1,fix,4,7,uniform,win
1,0,1,10.0,11.0,"[[[-0.010402418712398161, 0.013333949566327224...",1,fix,9,4,uniform,lose
2,1,0,9.5,11.5,"[[[0.09935288477605207, 0.013333949566327224],...",1,fix,1,9,uniform,lose
3,0,0,9.0,11.0,"[[[0.09935288477605207, 0.013333949566327224],...",1,fix,2,1,uniform,lose
4,1,1,9.5,11.5,"[[[0.09935288477605207, 0.013333949566327224],...",1,fix,9,3,uniform,win
...,...,...,...,...,...,...,...,...,...,...,...
265,0,0,69.0,87.0,"[[[0.4644159822024744, 0.013333949566327224], ...",4,mix,4,6,uniform,win
266,0,0,69.5,87.5,"[[[0.4644159822024744, 0.013333949566327224], ...",4,mix,6,8,high,win
267,1,0,69.0,88.0,"[[[0.4644159822024744, 0.013333949566327224], ...",4,mix,3,5,high,lose
268,0,0,69.5,88.5,"[[[0.4644159822024744, 0.013333949566327224], ...",4,mix,2,4,uniform,win


# total reward

In [10]:
num_subplots = len(df_participants)
trial_num = len(df_participants[0])

fig, axes = plt.subplots(nrows=7, ncols=6, figsize=(15, 18))
axes = axes.flatten()



labels = ['participants','wsls','greedy', 'softmax',  'dualQ', 'risk-sensitive']
colors = ['black','#E69F00', '#56B4E9', '#009E73', "#CEC10F", "#065B8D"]


for i in range(num_subplots):
    ax = axes[i]

    ax.plot(range(trial_num), (df_participants[i]['totalReward'] - df_participants[i]['totalReward'].iloc[0] )+10, label=labels[0], color=colors[0], linewidth=1.5)
    ax.plot(range(trial_num), df_wsls[i]['model_total_reward'], label=labels[1], color=colors[1], linewidth=0.5)
    ax.plot(range(trial_num), df_greedy[i]['model_total_reward'], label=labels[2], color=colors[2], linewidth=0.5)
    ax.plot(range(trial_num), df_softmax[i]['model_total_reward'], label=labels[3], color=colors[3], linewidth=0.5)
    ax.plot(range(trial_num), df_dualQ[i]['model_total_reward'], label=labels[4], color=colors[4], linewidth=0.5)
    ax.plot(range(trial_num), df_rs[i]['model_total_reward'], label=labels[5], color=colors[5], linewidth=0.5)


    ax.set_title(f'participant {i+1}' , fontsize=14, fontweight='bold')
    ax.set_xlabel('trials')
    ax.set_ylabel('total reward')
    # ax.legend(frameon=False)

    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

for j in range(num_subplots, len(axes)):
    fig.delaxes(axes[j])
    
fig.suptitle('total rewards', fontsize=22, fontweight='bold', y=1)


# Add colored legend text under the title
x_start = 0.1
x_spacing = 0.14
y_pos = 0.975  # Slightly below suptitle

for idx, (label, color) in enumerate(zip(labels, colors)):
    fig.text(x_start + idx * x_spacing, y_pos, label, fontsize=14, color=color, fontweight='bold')




plt.tight_layout(rect=[0, 0, 1, 1]) 

output_dir = pathlib.Path(output_dir).resolve()
pdf_path = output_dir / "total_reward.pdf"
svg_path = output_dir / "total_reward.svg"

fig.savefig(pdf_path)
fig.savefig(svg_path, format="svg", dpi=1200, bbox_inches="tight", transparent=True)

# plt.close()


# total reward score

In [11]:
# List of model DataFrames
model_dfs = [df_wsls, df_greedy, df_softmax, df_dualQ, df_rs]
model_names = labels[1:]  # Exclude 'participants'

num_participants = len(df_participants)
num_models = len(model_dfs)

# Store scores: rows=models, cols=participants
score_matrix = np.zeros((num_models, num_participants))

for model_idx, df_model in enumerate(model_dfs):
    for p_idx in range(num_participants):
        # Get trajectories
        y_true = (df_participants[p_idx]['totalReward'] - df_participants[p_idx]['totalReward'].iloc[0]) + 10
        y_pred = df_model[p_idx]['model_total_reward']

        # Ensure lengths match
        if len(y_true) != len(y_pred):
            raise ValueError(f"Length mismatch at model {model_names[model_idx]} and participant {p_idx+1}")

        # Compute correlation (use nan_to_num in case of nan), and RMSE
        corr, _ = pearsonr(y_true, y_pred)
        corr = np.nan_to_num(corr)
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))

        # Compute score
        score = abs(corr) / rmse if rmse != 0 else 0
        score_matrix[model_idx, p_idx] = score

# Plot heatmap
fig, ax = plt.subplots(figsize=(14, 4))
sns.heatmap(score_matrix, cmap='viridis', annot=False, cbar=True,
            xticklabels=[f'p{i+1}' for i in range(num_participants)],
            yticklabels=model_names, ax=ax)

ax.set_title('models-participants fit score', fontsize=16, fontweight='bold')
ax.set_xlabel('participants', fontsize=14)
ax.set_ylabel('models', fontsize=14)

plt.tight_layout()

# Save figure
heatmap_pdf = output_dir / "total_reward_model_fit_score_heatmap.pdf"
heatmap_svg = output_dir / "total_reward_model_fit_score_heatmap.svg"
fig.savefig(heatmap_pdf)
fig.savefig(heatmap_svg, format="svg", dpi=1200, bbox_inches="tight", transparent=True)



In [12]:

# Color palette for models
bar_colors = ['#E69F00', '#56B4E9', '#009E73', "#CEC10F", "#065B8D"]

# Step 1: Convert score_matrix to long-form DataFrame
score_data = []

for model_idx, model_name in enumerate(model_names):
    for p_idx in range(num_participants):
        score = score_matrix[model_idx, p_idx]
        score_data.append({'Model': model_name, 'Score': score})

score_df = pd.DataFrame(score_data)

# Step 2: Plot
fig, ax = plt.subplots(figsize=(4, 4))

# Draw boxplot with thinner borders (0.5 pt ≈ 0.666 px)
sns.boxplot(data=score_df, x='Model', y='Score', color='none',
            width=0.4, 
            boxprops=dict(facecolor='none', edgecolor='black', linewidth=0.666),
            whiskerprops=dict(color='black', linewidth=0.666),
            capprops=dict(color='black', linewidth=0.666),
            medianprops=dict(color='black', linewidth=0.666),
            fliersize=0,
            ax=ax)

# Plot colored scatter points per model (no edge color)
for i, model_name in enumerate(model_names):
    model_scores = score_df[score_df['Model'] == model_name]['Score']
    x_vals = np.random.normal(loc=i, scale=0.08, size=len(model_scores))  # jitter
    ax.scatter(x_vals, model_scores, color=bar_colors[i], s=10, alpha=0.8, edgecolors='none')

# Final plot settings
ax.set_title("total reward similarity", fontsize=15, fontweight='bold')
ax.set_ylabel("score", fontsize=13)
ax.set_xlabel(" ", fontsize=13)

# Remove top/right spines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Make xtick marks (small lines) 0.5 pt (≈ 0.666 px)
ax.tick_params(axis='x', length=3, width=0.666)  # small tick lines
ax.tick_params(axis='y', width=0.666)

plt.tight_layout()

# Save figure
boxplot_pdf = output_dir / "total_reward_model_fit_score_boxplot.pdf"
boxplot_svg = output_dir / "total_reward_model_fit_score_boxplot.svg"
fig.savefig(boxplot_pdf)
fig.savefig(boxplot_svg, format="svg", dpi=1200, bbox_inches="tight", transparent=True)

In [13]:
bar_colors = ['#E69F00', '#56B4E9', '#009E73', "#CEC10F", "#065B8D"]

# Step 1: Assign each participant to the model with the highest score
best_model_indices = np.argmax(score_matrix, axis=0)
assigned_counts = np.bincount(best_model_indices, minlength=len(model_names))

# Step 2: Sort by count (descending)
sorted_indices = np.argsort(-assigned_counts)  # negative for descending

sorted_model_names = [model_names[i] for i in sorted_indices]
sorted_counts = assigned_counts[sorted_indices]
sorted_colors = [bar_colors[i] for i in sorted_indices]

# Step 3: Create sorted histogram
fig, ax = plt.subplots(figsize=(3, 3))
bars = ax.bar(sorted_model_names, sorted_counts, color=sorted_colors)

# Annotate counts
for bar in bars:
    height = bar.get_height()
    ax.annotate(f'{height}',
                xy=(bar.get_x() + bar.get_width() / 2, height),
                xytext=(0, 3),
                textcoords="offset points",
                ha='center', va='bottom', fontsize=6)

ax.set_ylabel("number of participants", fontsize=8)
ax.set_title("total reward similarity", fontsize=10, fontweight='bold')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.xticks(rotation=20)
plt.tight_layout()

# Save
fig.savefig(output_dir / "total_reward_model_fit_score_hist.pdf")
fig.savefig(output_dir / "total_reward_model_fit_score_hist.svg", format="svg", dpi=1200, bbox_inches="tight", transparent=True)

In [14]:

trial_num = len(df_participants[0])

# Convert all reward trajectories to NumPy arrays (as float)
participant_rewards = np.stack([
    ((df['totalReward'] - df['totalReward'].iloc[0]) + 10).astype(float).values for df in df_participants
])
wsls_rewards = np.stack([df['model_total_reward'].astype(float).values for df in df_wsls])
greedy_rewards = np.stack([df['model_total_reward'].astype(float).values for df in df_greedy])
softmax_rewards = np.stack([df['model_total_reward'].astype(float).values for df in df_softmax])
dualQ_rewards = np.stack([df['model_total_reward'].astype(float).values for df in df_dualQ])
rs_rewards = np.stack([df['model_total_reward'].astype(float).values for df in df_rs])

# Compute mean and SEM manually
def mean_and_sem(data):
    mean = np.mean(data, axis=0)
    sem_val = np.std(data, axis=0, ddof=1) / np.sqrt(data.shape[0])
    return mean, sem_val

data_groups = [participant_rewards, wsls_rewards, greedy_rewards, softmax_rewards, dualQ_rewards, rs_rewards]
labels = ['participants', 'wsls', 'greedy', 'softmax', 'dualQ', 'risk-sensitive']
colors = ['black', '#E69F00', '#56B4E9', '#009E73', "#CEC10F", "#065B8D"]

# Plot
fig, ax = plt.subplots(figsize=(5, 5))

for data, label, color in zip(data_groups, labels, colors):
    mean_vals, sem_vals = mean_and_sem(data)
    ax.plot(range(trial_num), mean_vals, label=label, color=color, linewidth=2.5)
    ax.fill_between(range(trial_num), mean_vals - sem_vals, mean_vals + sem_vals, color=color, alpha=0.3)

# Style
ax.set_title("mean total reward across participants", fontsize=12, fontweight='bold')
ax.set_xlabel("trials", fontsize=14)
ax.set_ylabel("total reward", fontsize=14)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)


# Custom legend (stacked vertically inside top-left corner)
x_pos = 0.2   # left padding
y_start = 0.9 # top position
y_spacing = 0.045  # vertical spacing between labels

for idx, (label, color) in enumerate(zip(labels, colors)):
    fig.text(x_pos, y_start - idx * y_spacing, label,
             fontsize=10, color=color, fontweight='bold',
             ha='left', va='top')


plt.tight_layout(rect=[0, 0, 1, 1])

# Save

output_dir = pathlib.Path(output_dir).resolve()
pdf_path = output_dir / "total_reward_across_all.pdf"
svg_path = output_dir / "total_reward_across_all.svg"

fig.savefig(pdf_path)
fig.savefig(svg_path, format="svg", dpi=1200, bbox_inches="tight", transparent=True)

# plt.close()


# accuracy

In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

def compute_win_rates_by_block_type(df_list, x_labels=['uniform', 'low', 'high']):
    win_rates_mix = []
    win_rates_non_mix = []

    for df in df_list:
        df['is_win'] = df['outcome'].apply(lambda x: 1 if x == 'win' else 0)

        df_mix = df[df['block_type'] == 'mix']
        df_non_mix = df[df['block_type'] != 'mix']

        if not df_mix.empty:
            total_trials_mix = df_mix.groupby('distribution').size().reset_index(name='total_trials')
            wins_mix = df_mix.groupby('distribution')['is_win'].sum().reset_index(name='wins')
            win_rate_mix = pd.merge(total_trials_mix, wins_mix, on='distribution')
            win_rate_mix['win_rate'] = win_rate_mix['wins'] / win_rate_mix['total_trials']
            win_rates_mix.append(win_rate_mix[['distribution', 'win_rate']])

        if not df_non_mix.empty:
            total_trials_non_mix = df_non_mix.groupby('distribution').size().reset_index(name='total_trials')
            wins_non_mix = df_non_mix.groupby('distribution')['is_win'].sum().reset_index(name='wins')
            win_rate_non_mix = pd.merge(total_trials_non_mix, wins_non_mix, on='distribution')
            win_rate_non_mix['win_rate'] = win_rate_non_mix['wins'] / win_rate_non_mix['total_trials']
            win_rates_non_mix.append(win_rate_non_mix[['distribution', 'win_rate']])

    def compute_group_stats(win_rates_list):
        combined = pd.concat(win_rates_list)
        mean = combined.groupby('distribution')['win_rate'].mean().reset_index()
        std = combined.groupby('distribution')['win_rate'].std().reset_index()
        stats = mean.merge(std, on='distribution', suffixes=('_mean', '_std'))
        stats['distribution'] = pd.Categorical(stats['distribution'], categories=x_labels, ordered=True)
        return stats.sort_values(by='distribution').reset_index(drop=True)

    return compute_group_stats(win_rates_mix), compute_group_stats(win_rates_non_mix), win_rates_mix, win_rates_non_mix

# Compute win rates
mean_win_rates_df_participants_mix, mean_win_rates_df_participants_non_mix, win_rates_mix, win_rates_non_mix = compute_win_rates_by_block_type(df_participants)
mean_win_rates_df_wsls_mix, mean_win_rates_df_wsls_non_mix, win_rates_wsls_mix, win_rates_wsls_non_mix = compute_win_rates_by_block_type(df_wsls)
mean_win_rates_df_greedy_mix, mean_win_rates_df_greedy_non_mix, win_rates_greedy_mix, win_rates_greedy_non_mix = compute_win_rates_by_block_type(df_greedy)
mean_win_rates_df_softmax_mix, mean_win_rates_df_softmax_non_mix, win_rates_softmax_mix, win_rates_softmax_non_mix = compute_win_rates_by_block_type(df_softmax)
mean_win_rates_df_dualQ_mix, mean_win_rates_df_dualQ_non_mix, win_rates_dualQ_mix, win_rates_dualQ_non_mix = compute_win_rates_by_block_type(df_dualQ)
mean_win_rates_df_rs_mix, mean_win_rates_df_rs_non_mix, win_rates_rs_mix, win_rates_rs_non_mix = compute_win_rates_by_block_type(df_rs)

def compute_sem(win_rates_list):
    combined = pd.concat(win_rates_list)
    sem = combined.groupby('distribution')['win_rate'].sem().reset_index()
    return sem

# Organize data
mean_dfs_fix = [mean_win_rates_df_participants_non_mix, mean_win_rates_df_wsls_non_mix, mean_win_rates_df_greedy_non_mix, mean_win_rates_df_softmax_non_mix, mean_win_rates_df_dualQ_non_mix, mean_win_rates_df_rs_non_mix]
mean_dfs_mix = [mean_win_rates_df_participants_mix, mean_win_rates_df_wsls_mix, mean_win_rates_df_greedy_mix, mean_win_rates_df_softmax_mix, mean_win_rates_df_dualQ_mix, mean_win_rates_df_rs_mix]

sem_dfs_fix = [compute_sem(lst) for lst in [win_rates_non_mix, win_rates_wsls_non_mix, win_rates_greedy_non_mix, win_rates_softmax_non_mix, win_rates_dualQ_non_mix, win_rates_rs_non_mix]]
sem_dfs_mix = [compute_sem(lst) for lst in [win_rates_mix, win_rates_wsls_mix, win_rates_greedy_mix, win_rates_softmax_mix, win_rates_dualQ_mix, win_rates_rs_mix]]

labels = ['participants', 'wsls', 'greedy', 'softmax', 'dualQ', 'risk-sensitive']
colors = ['black', '#E69F00', '#56B4E9', '#009E73', "#CEC10F", "#065B8D"]

# Plotting
x_labels = ['uniform', 'low', 'high']
bar_width = 0.12
x = np.arange(len(x_labels))

fig, axes = plt.subplots(1, 2, figsize=(12, 4), sharey=True)
bars = []

for ax_idx, (ax, mean_dfs, sem_dfs, title) in enumerate(zip(
    axes,
    [mean_dfs_fix, mean_dfs_mix],
    [sem_dfs_fix, sem_dfs_mix],
    ['fix', 'mix']
)):

    for j, dist in enumerate(x_labels):  # for each distribution
        # Get participants bar first
        mean_p = mean_dfs[0][mean_dfs[0]['distribution'] == dist]['win_rate_mean'].values[0]
        sem_p = sem_dfs[0][sem_dfs[0]['distribution'] == dist]['win_rate'].values[0]
        bar = ax.bar(j, mean_p, width=bar_width, color=colors[0], yerr=sem_p, capsize=5, label=labels[0] if ax_idx == 0 and j == 0 else None)
        if ax_idx == 0 and j == 0:
            bars.append(bar[0])

        # Gather and sort all models except participants
        dist_vals = []
        for i in range(1, len(mean_dfs)):
            mean_val = mean_dfs[i][mean_dfs[i]['distribution'] == dist]['win_rate_mean'].values[0]
            sem_val = sem_dfs[i][sem_dfs[i]['distribution'] == dist]['win_rate'].values[0]
            dist_vals.append((labels[i], colors[i], mean_val, sem_val))

        dist_vals.sort(key=lambda x: x[2], reverse=True)  # sort by win_rate

        for k, (label, color, mean_val, sem_val) in enumerate(dist_vals):
            bar = ax.bar(j + (k + 1) * bar_width, mean_val, width=bar_width, color=color,
                         yerr=sem_val, capsize=5, label=label if ax_idx == 0 and j == 0 else None)
            if ax_idx == 0 and j == 0:
                bars.append(bar[0])

    ax.set_title(f'{title}', fontweight='bold')
    ax.set_xticks(np.arange(len(x_labels)) + ((len(mean_dfs)) / 2) * bar_width / 2)
    ax.set_xticklabels(x_labels)
    ax.set_ylabel('accuracy')
    ax.set_ylim(0, 1)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

fig.legend(bars, labels, loc='upper center', ncol=4, frameon=False, bbox_to_anchor=(0.5, 0.9))
plt.suptitle('accuracy by distribution', fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.88])

# Save figure


output_dir = pathlib.Path(output_dir).resolve()
pdf_path = output_dir / "accuracy_fix_mix.pdf"
svg_path = output_dir / "accuracy_fix_mix.svg"

fig.savefig(pdf_path)
fig.savefig(svg_path, format="svg", dpi=1200, bbox_inches="tight", transparent=True)


In [16]:

def compute_win_rates_by_block_type(df_list, x_labels=['uniform', 'low', 'high']):
    win_rates_mix = []
    win_rates_non_mix = []

    for df in df_list:
        df['is_win'] = df['outcome'].apply(lambda x: 1 if x == 'win' else 0)

        df_mix = df[df['block_type'] == 'mix']
        df_non_mix = df[df['block_type'] != 'mix']

        if not df_mix.empty:
            total_trials_mix = df_mix.groupby('distribution').size().reset_index(name='total_trials')
            wins_mix = df_mix.groupby('distribution')['is_win'].sum().reset_index(name='wins')
            win_rate_mix = pd.merge(total_trials_mix, wins_mix, on='distribution')
            win_rate_mix['win_rate'] = win_rate_mix['wins'] / win_rate_mix['total_trials']
            win_rates_mix.append(win_rate_mix[['distribution', 'win_rate']])

        if not df_non_mix.empty:
            total_trials_non_mix = df_non_mix.groupby('distribution').size().reset_index(name='total_trials')
            wins_non_mix = df_non_mix.groupby('distribution')['is_win'].sum().reset_index(name='wins')
            win_rate_non_mix = pd.merge(total_trials_non_mix, wins_non_mix, on='distribution')
            win_rate_non_mix['win_rate'] = win_rate_non_mix['wins'] / win_rate_non_mix['total_trials']
            win_rates_non_mix.append(win_rate_non_mix[['distribution', 'win_rate']])

    def compute_group_stats(win_rates_list):
        combined = pd.concat(win_rates_list)
        mean = combined.groupby('distribution')['win_rate'].mean().reset_index()
        std = combined.groupby('distribution')['win_rate'].std().reset_index()
        stats = mean.merge(std, on='distribution', suffixes=('_mean', '_std'))
        stats['distribution'] = pd.Categorical(stats['distribution'], categories=x_labels, ordered=True)
        return stats.sort_values(by='distribution').reset_index(drop=True)

    return compute_group_stats(win_rates_mix), compute_group_stats(win_rates_non_mix), win_rates_mix, win_rates_non_mix


def compute_sem(win_rates_list):
    combined = pd.concat(win_rates_list)
    sem = combined.groupby('distribution')['win_rate'].sem().reset_index()
    return sem

def get_sorted_indices(mean_dfs):
    avg_accuracies = [df['win_rate_mean'].mean() for df in mean_dfs[1:]]
    sorted_indices = np.argsort(avg_accuracies)[::-1]
    return sorted_indices

def reorder_lists(mean_dfs, sem_dfs, labels, colors):
    sorted_idx = get_sorted_indices(mean_dfs)
    reordered_mean = [mean_dfs[0]] + [mean_dfs[i + 1] for i in sorted_idx]
    reordered_sem  = [sem_dfs[0]]  + [sem_dfs[i + 1] for i in sorted_idx]
    reordered_lbls = [labels[0]]   + [labels[i + 1] for i in sorted_idx]
    reordered_clrs = [colors[0]]   + [colors[i + 1] for i in sorted_idx]
    return reordered_mean, reordered_sem, reordered_lbls, reordered_clrs

def combine_fix_mix_means_and_sems(mean_fix, mean_mix, sem_fix, sem_mix):
    combined_means = []
    combined_sems = []
    for df_fix, df_mix, se_fix, se_mix in zip(mean_fix, mean_mix, sem_fix, sem_mix):
        df_comb = df_fix.copy()
        df_comb['win_rate_mean'] = (df_fix['win_rate_mean'] + df_mix['win_rate_mean']) / 2
        sem_comb = se_fix.copy()
        sem_comb['win_rate'] = np.sqrt((se_fix['win_rate']**2 + se_mix['win_rate']**2) / 2)
        combined_means.append(df_comb)
        combined_sems.append(sem_comb)
    return combined_means, combined_sems



mean_win_rates_df_participants_mix, mean_win_rates_df_participants_non_mix, win_rates_mix, win_rates_non_mix = compute_win_rates_by_block_type(df_participants)
mean_win_rates_df_wsls_mix, mean_win_rates_df_wsls_non_mix, win_rates_wsls_mix, win_rates_wsls_non_mix = compute_win_rates_by_block_type(df_wsls)
mean_win_rates_df_greedy_mix, mean_win_rates_df_greedy_non_mix, win_rates_greedy_mix, win_rates_greedy_non_mix = compute_win_rates_by_block_type(df_greedy)
mean_win_rates_df_softmax_mix, mean_win_rates_df_softmax_non_mix, win_rates_softmax_mix, win_rates_softmax_non_mix = compute_win_rates_by_block_type(df_softmax)
mean_win_rates_df_dualQ_mix, mean_win_rates_df_dualQ_non_mix, win_rates_dualQ_mix, win_rates_dualQ_non_mix = compute_win_rates_by_block_type(df_dualQ)
mean_win_rates_df_rs_mix, mean_win_rates_df_rs_non_mix, win_rates_rs_mix, win_rates_rs_non_mix = compute_win_rates_by_block_type(df_rs)

mean_dfs_fix = [mean_win_rates_df_participants_non_mix, mean_win_rates_df_wsls_non_mix, mean_win_rates_df_greedy_non_mix, mean_win_rates_df_softmax_non_mix, mean_win_rates_df_dualQ_non_mix, mean_win_rates_df_rs_non_mix]
mean_dfs_mix = [mean_win_rates_df_participants_mix, mean_win_rates_df_wsls_mix, mean_win_rates_df_greedy_mix, mean_win_rates_df_softmax_mix, mean_win_rates_df_dualQ_mix, mean_win_rates_df_rs_mix]

sem_dfs_fix = [compute_sem(lst) for lst in [win_rates_non_mix, win_rates_wsls_non_mix, win_rates_greedy_non_mix, win_rates_softmax_non_mix, win_rates_dualQ_non_mix, win_rates_rs_non_mix]]
sem_dfs_mix = [compute_sem(lst) for lst in [win_rates_mix, win_rates_wsls_mix, win_rates_greedy_mix, win_rates_softmax_mix, win_rates_dualQ_mix, win_rates_rs_mix]]

labels = ['participants', 'wsls', 'greedy', 'softmax', 'dualQ', 'risk-sensitive']
colors = ['black', '#E69F00', '#56B4E9', '#009E73', "#CEC10F", "#065B8D"]

re_mean_fix, re_sem_fix, re_labels_fix, re_colors_fix = reorder_lists(mean_dfs_fix, sem_dfs_fix, labels, colors)
re_mean_mix, re_sem_mix, _, _ = reorder_lists(mean_dfs_mix, sem_dfs_mix, labels, colors)

combined_mean_dfs, combined_sem_dfs = combine_fix_mix_means_and_sems(re_mean_fix, re_mean_mix, re_sem_fix, re_sem_mix)

# ---- Step 3: final combined plot ----

x_labels = ['uniform', 'low', 'high']
x = np.arange(len(x_labels))
bar_width = 0.12

fig, ax = plt.subplots(figsize=(10, 5))
bars = []

for i, (mean_df, sem_df, color, label) in enumerate(zip(combined_mean_dfs, combined_sem_dfs, re_colors_fix, re_labels_fix)):
    means = mean_df['win_rate_mean'].values
    errors = sem_df['win_rate'].values
    bar = ax.bar(x + i * bar_width, means, width=bar_width, label=label, color=color, yerr=errors, capsize=5)
    bars.append(bar[0])

ax.set_title('accuracy by distribution', fontweight='bold')
ax.set_xticks(x + (len(combined_mean_dfs) * bar_width) / 2 - bar_width / 2)
ax.set_xticklabels(x_labels)
ax.set_ylabel('accuracy')
ax.set_ylim(0, 1)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

fig.legend(bars, re_labels_fix, loc='upper center', ncol=4, frameon=False, bbox_to_anchor=(0.5, 0.92))
plt.tight_layout(rect=[0, 0, 1, 1])


output_dir = pathlib.Path(output_dir).resolve()
pdf_path = output_dir / "accuracy.pdf"
svg_path = output_dir / "accuracy.svg"

fig.savefig(pdf_path)
fig.savefig(svg_path, format="svg", dpi=1200, bbox_inches="tight", transparent=True)



# arrow up down percentage

In [17]:

# Color and dist setup
distributions = ['uniform', 'low', 'high']
colors = {'uniform': '#808080', 'low': '#ff7f0e', 'high': '#2ca02c'}
models = [
    (df_participants, 'participants', 'choice'),
    (df_wsls, 'wsls', 'model_choices'),
    (df_greedy, 'greedy', 'model_choices'),
    (df_softmax, 'softmax', 'model_choices'),
    (df_dualQ, 'dualQ', 'model_choices'),
    (df_rs, 'risk-sensitive', 'model_choices'),
]

# Simplified 3-parameter sigmoid function
def sigmoid3(x, L, x0, k):
    return L / (1 + np.exp(-k * (x - x0)))

# Compute arrow-up percentages by card
def compute_arrowup_stats(dataframes, dist, block_type, choice_col):
    group_results = []

    for df in dataframes:
        sub_df = df[(df['distribution'] == dist) & (df['block_type'] == block_type)].copy()
        sub_df['group'] = sub_df['myCard'].astype(str)

        if choice_col != 'choice':
            sub_df[choice_col] = sub_df[choice_col].map({1: 'arrowup', 0: 'arrowdown'})

        group_counts = sub_df.groupby(['group', choice_col], observed=True)[choice_col].count().unstack(fill_value=0)
        group_totals = group_counts.sum(axis=1)
        group_percentages = (group_counts.T / group_totals).T * 100
        group_percentages['group'] = group_percentages.index
        group_results.append(group_percentages.reset_index(drop=True))

    if not group_results:
        return None, None, None

    combined_df = pd.concat(group_results)
    mean_df = combined_df.groupby('group', observed=True).mean().reset_index()
    sem_df = combined_df.groupby('group', observed=True).sem().reset_index()
    return mean_df, sem_df, mean_df['group']

# Plotting
fig, axes = plt.subplots(2, 6, figsize=(20, 6), dpi=130, sharey=True)

for col, (df_group, label, choice_col) in enumerate(models):
    for row, block_type in enumerate(['fix', 'mix']):
        ax = axes[row, col]

        for dist in distributions:
            mean_df, sem_df, groups = compute_arrowup_stats(df_group, dist, block_type, choice_col)
            if mean_df is None:
                continue

            arrow_up = mean_df.get('arrowup', 0).values
            sem_up = sem_df.get('arrowup', 0).values
            x = np.arange(len(groups))
            color = colors[dist]

            # Error bars
            ax.errorbar(x, arrow_up, yerr=sem_up, fmt='o', color=color,
                        capsize=1, capthick=0.2, markersize=1.5, elinewidth=0.3, label=dist)

            with warnings.catch_warnings():
                warnings.simplefilter("error", OptimizeWarning)
                if len(np.unique(arrow_up)) > 2:
                    try:
                        p0 = [max(arrow_up), np.median(x), 1]  # 3-parameter initial guess
                        popt, _ = curve_fit(sigmoid3, x, arrow_up, p0, maxfev=800000)
                        x_smooth = np.linspace(x.min(), x.max(), 200)
                        y_smooth = sigmoid3(x_smooth, *popt)
                        ax.plot(x_smooth, y_smooth, color=color, linewidth=1, alpha=0.6, linestyle='--')
                    except (RuntimeError, OptimizeWarning, ValueError) as e:
                        print(f"Fit failed for {label} | block: {block_type} | dist: {dist}: {e}")

        ax.set_xticks(x)
        ax.set_xticklabels(groups)
        ax.set_ylim(0, 100)

        if row == 1:
            ax.set_xlabel("myCard", fontsize=10)
        if col == 0:
            ax.set_ylabel(f"{block_type}", fontsize=12, fontweight='bold')

        ax.set_title(label, fontsize=11, weight='bold')
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)

# Legend only once – below the plots
handles = [plt.Line2D([0], [0], color=colors[d], lw=2, label=d) for d in distributions]
fig.legend(
    handles=handles,
    loc='lower center',
    bbox_to_anchor=(0.5, 0.9), 
    ncol=3,
    frameon=False
)

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.suptitle('arrow up percentage', fontsize=16, fontweight='bold')

# Save
output_dir = pathlib.Path(output_dir).resolve()
pdf_path = output_dir / "arrowup_percentage.pdf"
svg_path = output_dir / "arrowup_percentage.svg"

fig.savefig(pdf_path)
fig.savefig(svg_path, format="svg", dpi=1200, bbox_inches="tight", transparent=True)

# delta q and arrowRT

In [18]:
# arrowRT_participants = []

# for df in df_participants:
#     arrowRT_normalized = np.array(df['arrowRT'], dtype=float) / np.mean(df['arrowRT'])
#     arrowRT_participants.append(arrowRT_normalized)


# arrowRT_participants = np.array(arrowRT_participants)


# xticklabels = [f'p{i+1}' for i in range(num_participants)]
# yticklabels = ['uniform', 'low', 'high']


# #######################################################################################################################################





# delta_q_val_1_both = []
# delta_q_val_2_both = []
# delta_q_val_3_both = []
# delta_q_val_4_both = []
# delta_q_val_5_both = []
# delta_q_val_6_both = []
# delta_q_val_7_both = []
# delta_q_val_8_both = []
# delta_q_val_9_both = []

# for df in df_rs:
    

#     delta_1_both = []
#     delta_2_both = []
#     delta_3_both = []
#     delta_4_both = []
#     delta_5_both = []
#     delta_6_both = []
#     delta_7_both = []
#     delta_8_both = []
#     delta_9_both = []
    
#     temp_df = df['q_val']
#     for q_vals in temp_df:
        
#         temp_vec2 = np.array(ast.literal_eval(q_vals)) 
#         delta = temp_vec2[:, :, actions["arrowup"]].squeeze() - temp_vec2[:, :, actions["arrowdown"]].squeeze()
#         delta_1_both.append(delta[0][:])
#         delta_2_both.append(delta[1][:])
#         delta_3_both.append(delta[2][:])
#         delta_4_both.append(delta[3][:])
#         delta_5_both.append(delta[4][:])
#         delta_6_both.append(delta[5][:])
#         delta_7_both.append(delta[6][:])
#         delta_8_both.append(delta[7][:])
#         delta_9_both.append(delta[8][:])
        
        

#     delta_q_val_1_both.append(delta_1_both)
#     delta_q_val_2_both.append(delta_2_both)
#     delta_q_val_3_both.append(delta_3_both)
#     delta_q_val_4_both.append(delta_4_both)
#     delta_q_val_5_both.append(delta_5_both)
#     delta_q_val_6_both.append(delta_6_both)
#     delta_q_val_7_both.append(delta_7_both)
#     delta_q_val_8_both.append(delta_8_both)
#     delta_q_val_9_both.append(delta_9_both)
        
    



# delta_q_val_1_both = np.abs(np.array(delta_q_val_1_both))
# delta_q_val_2_both = np.abs(np.array(delta_q_val_2_both))
# delta_q_val_3_both = np.abs(np.array(delta_q_val_3_both))
# delta_q_val_4_both = np.abs(np.array(delta_q_val_4_both))
# delta_q_val_5_both = np.abs(np.array(delta_q_val_5_both))
# delta_q_val_6_both = np.abs(np.array(delta_q_val_6_both))
# delta_q_val_7_both = np.abs(np.array(delta_q_val_7_both))
# delta_q_val_8_both = np.abs(np.array(delta_q_val_8_both))
# delta_q_val_9_both = np.abs(np.array(delta_q_val_9_both))

# # Create storage for correlations and p-values for each RT type
# correlations_arrow = np.zeros((num_participants, 9, len(distributions_map)))
# pvals_arrow = np.zeros((num_participants, 9, len(distributions_map)))

# # --- Compute Correlations (Arrow RT) ---
# for i in range(num_participants):
#     # number 1
#     corr = np.corrcoef(arrowRT_participants[i, :],
#                        delta_q_val_1_both[i, :, distributions_map["uniform"]])[0, 1]
#     correlations_arrow[i, 0, distributions_map["uniform"]] = corr
#     _, pval = pearsonr(arrowRT_participants[i, :],
#                        delta_q_val_1_both[i, :, distributions_map["uniform"]])
#     pvals_arrow[i, 0, distributions_map["uniform"]] = pval

#     corr = np.corrcoef(arrowRT_participants[i, :],
#                        delta_q_val_1_both[i, :, distributions_map["low"]])[0, 1]
#     correlations_arrow[i, 0, distributions_map["low"]] = corr
#     _, pval = pearsonr(arrowRT_participants[i, :],
#                        delta_q_val_1_both[i, :, distributions_map["low"]])
#     pvals_arrow[i, 0, distributions_map["low"]] = pval

#     corr = np.corrcoef(arrowRT_participants[i, :],
#                        delta_q_val_1_both[i, :, distributions_map["high"]])[0, 1]
#     correlations_arrow[i, 0, distributions_map["high"]] = corr
#     _, pval = pearsonr(arrowRT_participants[i, :],
#                        delta_q_val_1_both[i, :, distributions_map["high"]])
#     pvals_arrow[i, 0, distributions_map["high"]] = pval

#     # Repeat for numbers 2 through 9
#     for num in range(2, 10):
#         delta_q_val_both = eval(f"delta_q_val_{num}_both")
#         corr = np.corrcoef(arrowRT_participants[i, :],
#                            delta_q_val_both[i, :, distributions_map["uniform"]])[0, 1]
#         correlations_arrow[i, num - 1, distributions_map["uniform"]] = corr
#         _, pval = pearsonr(arrowRT_participants[i, :],
#                            delta_q_val_both[i, :, distributions_map["uniform"]])
#         pvals_arrow[i, num - 1, distributions_map["uniform"]] = pval

#         corr = np.corrcoef(arrowRT_participants[i, :],
#                            delta_q_val_both[i, :, distributions_map["low"]])[0, 1]
#         correlations_arrow[i, num - 1, distributions_map["low"]] = corr
#         _, pval = pearsonr(arrowRT_participants[i, :],
#                            delta_q_val_both[i, :, distributions_map["low"]])
#         pvals_arrow[i, num - 1, distributions_map["low"]] = pval

#         corr = np.corrcoef(arrowRT_participants[i, :],
#                            delta_q_val_both[i, :, distributions_map["high"]])[0, 1]
#         correlations_arrow[i, num - 1, distributions_map["high"]] = corr
#         _, pval = pearsonr(arrowRT_participants[i, :],
#                            delta_q_val_both[i, :, distributions_map["high"]])
#         pvals_arrow[i, num - 1, distributions_map["high"]] = pval

# # ----------------------------------------------------------
# # ----------------------------------------------------------
# fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(7, 8))  # 3x1 layout
# labels = ['uniform', 'low', 'high']

# for row_idx, dist_label in enumerate(labels):
#     # ARROW
#     ax_arrow = axes[row_idx]
#     data_arrow = correlations_arrow[:, :, distributions_map[dist_label]].T

#     # Make an annotation array (same shape) for significance
#     annots_arrow = np.empty((9, num_participants), dtype=object)
#     for r in range(9):
#         for c in range(num_participants):
#             if pvals_arrow[c, r, distributions_map[dist_label]] < 0.05:
#                 annots_arrow[r, c] = '*'
#             else:
#                 annots_arrow[r, c] = ''

#     sns.heatmap(
#         data_arrow,
#         ax=ax_arrow,
#         cmap='coolwarm',
#         linewidths=0.5,
#         xticklabels=[f'p{i+1}' for i in range(num_participants)],
#         yticklabels=list(range(1, 10)),
#         cbar=True,
#         vmin=-0.4, vmax=0.4,
#         annot=annots_arrow, fmt=""
#     )
#     ax_arrow.set_title(f"{dist_label}")
#     if row_idx == 2:
#         ax_arrow.set_xlabel("participants", fontsize=11, fontweight='bold')

# fig.suptitle('correlation between \n|q(up)-q(down)| and flip RT', 
#              fontsize=16, fontweight='bold', y=0.98)
# plt.tight_layout()


# Save
# output_dir = pathlib.Path(output_dir).resolve()
# pdf_path = output_dir / "correlation_qVal_arrow_RT.pdf"
# svg_path = output_dir / "correlation_qVal_arrow_RT.svg"

# fig.savefig(pdf_path)
# fig.savefig(svg_path, format="svg", dpi=1200, bbox_inches="tight", transparent=True)






In [19]:

# def permutation_corr(x, y, n_permutations=1000, seed=42):
#     rng = np.random.default_rng(seed)
#     actual_corr, _ = pearsonr(x, y)
#     permuted_corrs = np.zeros(n_permutations)
    
#     for i in range(n_permutations):
#         y_perm = rng.permutation(y)
#         permuted_corrs[i], _ = pearsonr(x, y_perm)
    
#     pval = np.mean(np.abs(permuted_corrs) >= np.abs(actual_corr))
#     return actual_corr, pval

# # Your original values
# arrowRT_avg = np.mean(arrowRT_participants, axis=0)  # shape: (num_trials,)
# correlation_means = np.zeros((9, 3))
# pvals_means = np.zeros((9, 3))

# # Loop through numbers 1 to 9
# for num in range(1, 10):
#     delta_q = eval(f"delta_q_val_{num}_both")  # shape: (num_participants, num_trials, 3)
#     delta_q_mean = np.mean(delta_q, axis=0)  # shape: (num_trials, 3)
    
#     for dist_name, dist_idx in distributions_map.items():
#         delta_vals = delta_q_mean[:, dist_idx]  # shape: (num_trials,)
#         corr, pval = permutation_corr(arrowRT_avg, delta_vals, n_permutations=1000)
        
#         correlation_means[num - 1, dist_idx] = corr
#         pvals_means[num - 1, dist_idx] = pval

# # Plotting
# fig, ax = plt.subplots(figsize=(3, 6))
# labels = ['uniform', 'low', 'high']

# annot = np.empty((9, 3), dtype=object)
# for i in range(9):
#     for j in range(3):
#         annot[i, j] = '*' if pvals_means[i, j] < 0.05 else ''

# sns.heatmap(
#     correlation_means,
#     cmap='coolwarm',
#     annot=annot,
#     fmt='',
#     xticklabels=labels,
#     yticklabels=[str(i) for i in range(1, 10)],
#     linewidths=0.5,
#     vmin=-0.4, vmax=0.4,
#     cbar=True,
#     ax=ax
# )

# ax.set_xlabel("distribution", fontsize=11, fontweight='bold')
# ax.set_ylabel("card Number", fontsize=11, fontweight='bold')
# ax.set_title("correlation between\n |Q(up) - Q(down)| and RT", fontsize=13, fontweight='bold')
# plt.tight_layout()

# # Save
# output_dir = pathlib.Path(output_dir).resolve()
# pdf_path = output_dir / "correlation_avg_qVal_arrow_RT.pdf"
# svg_path = output_dir / "correlation_avg_qVal_arrow_RT.svg"
# fig.savefig(pdf_path)
# fig.savefig(svg_path, format="svg", dpi=1200, bbox_inches="tight", transparent=True)


In [20]:


arrowRT_participants = []

for df in df_participants:
    arrowRT_normalized = np.array(df['arrowRT'], dtype=float) / np.mean(df['arrowRT'])
    arrowRT_participants.append(arrowRT_normalized)

arrowRT_participants = np.array(arrowRT_participants)

xticklabels = [f'p{i+1}' for i in range(num_participants)]
yticklabels = ['uniform', 'low', 'high']

#######################################################################################################################################

def compute_delta_q_vals(df_model):
    deltas_all = [[] for _ in range(9)]

    for df in df_model:
        delta_cards = [[] for _ in range(9)]
        temp_df = df['q_val']

        for q_vals in temp_df:
            temp_vec2 = np.array(ast.literal_eval(q_vals)) 
            delta = temp_vec2[:, :, actions["arrowup"]].squeeze() - temp_vec2[:, :, actions["arrowdown"]].squeeze()
            for card_idx in range(9):
                delta_cards[card_idx].append(delta[card_idx][:])

        for i in range(9):
            deltas_all[i].append(delta_cards[i])

    return [np.abs(np.array(d)) for d in deltas_all]

def permutation_corr(x, y, n_permutations=1000, seed=42):
    rng = np.random.default_rng(seed)
    actual_corr, _ = pearsonr(x, y)
    permuted_corrs = np.zeros(n_permutations)
    
    for i in range(n_permutations):
        y_perm = rng.permutation(y)
        permuted_corrs[i], _ = pearsonr(x, y_perm)
    
    pval = np.mean(np.abs(permuted_corrs) >= np.abs(actual_corr))
    return actual_corr, pval

# List of models and their dataframes
model_dfs = [
    ('greedy', df_greedy),
    ('softmax', df_softmax),
    ('dualQ', df_dualQ),
    ('RS', df_rs)
]


# Prepare figure

import matplotlib.gridspec as gridspec

fig = plt.figure(figsize=(8, 4))
gs = gridspec.GridSpec(nrows=1, ncols=5, width_ratios=[1, 1, 1, 1, 0.05], wspace=0.3)
axes = [fig.add_subplot(gs[0, i]) for i in range(4)]
cbar_ax = fig.add_subplot(gs[0, 4])  # reserved for the shared colorbar

for ax, (model_name, df_model) in zip(axes, model_dfs):
    delta_q_vals = compute_delta_q_vals(df_model)

    arrowRT_avg = np.mean(arrowRT_participants, axis=0)
    correlation_means = np.zeros((9, 3))
    pvals_means = np.zeros((9, 3))

    for num in range(1, 10):
        delta_q = delta_q_vals[num - 1]
        delta_q_mean = np.mean(delta_q, axis=0)

        for dist_name, dist_idx in distributions_map.items():
            delta_vals = delta_q_mean[:, dist_idx]
            corr, pval = permutation_corr(arrowRT_avg, delta_vals, n_permutations=1000)
            correlation_means[num - 1, dist_idx] = corr
            pvals_means[num - 1, dist_idx] = pval

    annot = np.empty((9, 3), dtype=object)
    for i in range(9):
        for j in range(3):
            annot[i, j] = '*' if pvals_means[i, j] < 0.05 else ''

    sns.heatmap(
        correlation_means,
        cmap='coolwarm',
        annot=annot,
        fmt='',
        xticklabels=['uniform', 'low', 'high'],  
        yticklabels=[str(i) for i in range(1, 10)],
        linewidths=0.5,
        vmin=-0.4, vmax=0.4,
        cbar=(ax == axes[-1]),
        cbar_ax=cbar_ax if ax == axes[-1] else None,
        ax=ax
    )


    ax.set_title(model_name, fontsize=12, fontweight='bold')
    ax.set_xlabel("distribution", fontsize=11, fontweight='bold')
    ax.set_ylabel("card number" if ax == axes[0] else "")

plt.suptitle("|Q(up) - Q(down)| and RT correlation", fontsize=14, fontweight='bold')
plt.tight_layout(rect=[0, 0, 1, 0.95])

# Save
pdf_path = output_dir / "correlation_avg_qVal_arrow_RT.pdf"
svg_path = output_dir / "correlation_avg_qVal_arrow_RT.svg"
fig.savefig(pdf_path)
fig.savefig(svg_path, format="svg", dpi=1200, bbox_inches="tight", transparent=True)


# delta q vals changes

# delta q vals scatter plot